In [1]:
! pip3 install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import glob
import os
import boto3
import socket
from botocore.handlers import disable_signing
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn

In [4]:
if socket.gethostname() == 'Rohits-MBP':
    rootdir = '/Users/rohitchanne/Documents/capstone/data/data_parquet/'
else: 
    rootdir = '' # Enter your hone dir here

In [5]:
list_of_files = filter( os.path.isfile, glob.glob(rootdir + '*') )
files_with_size = [ file_path for file_path in list_of_files ]

In [6]:
dfs_parquet = {}
for file_path in files_with_size:
    if 'parquet' in file_path:
        file_name = file_path.split('/')[-1]
        df_name = file_name.split('_')[0]
        print(f'Reading Data File: {file_name}')       
        dfs_parquet[df_name] = pd.read_parquet(file_path, engine='pyarrow')

Reading Data File: Wednesday-21-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-23-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-22-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-16-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-28-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-14-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-15-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-01-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-02-03-2018_TrafficForML_CICFlowMeter_clean.parquet


In [25]:
dfs_parquet['Thuesday-20-02-2018'].drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP'], axis=1, inplace=True)

In [7]:
for k,df in dfs_parquet.items():
    print(k)
    df['is_allowed'] = df['Label'] == 'Benign'
    del df['Label']
    print(f'DF:{k}, Shape{df.shape}')

Wednesday-21-02-2018
DF:Wednesday-21-02-2018, Shape(1048575, 80)
Friday-23-02-2018
DF:Friday-23-02-2018, Shape(1048575, 80)
Thuesday-20-02-2018
DF:Thuesday-20-02-2018, Shape(7948748, 84)
Thursday-22-02-2018
DF:Thursday-22-02-2018, Shape(1048575, 80)
Friday-16-02-2018
DF:Friday-16-02-2018, Shape(1048574, 80)
Wednesday-28-02-2018
DF:Wednesday-28-02-2018, Shape(613071, 80)
Wednesday-14-02-2018
DF:Wednesday-14-02-2018, Shape(1048575, 80)
Thursday-15-02-2018
DF:Thursday-15-02-2018, Shape(1048575, 80)
Thursday-01-03-2018
DF:Thursday-01-03-2018, Shape(331100, 80)
Friday-02-03-2018
DF:Friday-02-03-2018, Shape(1048575, 80)


In [9]:
from sklearn.preprocessing import MinMaxScaler
cat_cols = ['Dst Port']

In [27]:
df_train = pd.concat([dfs_parquet['Friday-02-03-2018'], 
                      dfs_parquet['Friday-16-02-2018'], 
                      dfs_parquet['Friday-23-02-2018'],
                      dfs_parquet['Thursday-01-03-2018'],
                      dfs_parquet['Thursday-15-02-2018'],
                      dfs_parquet['Thursday-22-02-2018'],
                      dfs_parquet['Thuesday-20-02-2018'],
                      dfs_parquet['Wednesday-14-02-2018'],
                      dfs_parquet['Wednesday-21-02-2018']
                     ]
                    )
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_train['Timestamp'] = pd.to_datetime(df_train['Timestamp'])
df_train['Date'] = pd.to_datetime(df_train['Timestamp']).dt.date
df_train['TS_relative'] = (df_train['Timestamp'].astype(int) - 
                             pd.to_datetime(df_train['Date']).astype(int))/ 10**9
df_train = df_train.drop(['Timestamp'], axis = 1)
df_train = df_train.drop(['Date'], axis = 1)
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train.fillna(0, inplace=True)
df_train[cat_cols] = df_train[cat_cols].astype('category')
X_train = df_train.drop(['is_allowed'], axis = 1)
y_train = df_train['is_allowed']*1 

In [28]:
stdsc = MinMaxScaler()
stdsc.fit(X_train)
X_train_scaled = stdsc.transform(X_train)

In [54]:
iterations = 50

In [29]:
# df_test = pd.concat([dfs_parquet['Wednesday-14-02-2018'],
#                     dfs_parquet['Wednesday-21-02-2018'],
#                     dfs_parquet['Wednesday-28-02-2018']]
#                      )
df_test = dfs_parquet['Wednesday-28-02-2018']
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])
df_test['Date'] = pd.to_datetime(df_test['Timestamp']).dt.date
df_test['TS_relative'] = (df_test['Timestamp'].astype(int) - 
                             pd.to_datetime(df_test['Date']).astype(int))/ 10**9
df_test = df_test.drop(['Timestamp'], axis = 1)
df_test = df_test.drop(['Date'], axis = 1)
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test.fillna(0, inplace=True)
df_test[cat_cols] = df_test[cat_cols].astype('category')
X_test = df_test.drop(['is_allowed'], axis = 1)
y_test = df_test['is_allowed'].astype(int)*1

In [30]:
stdsc = MinMaxScaler()
stdsc.fit(X_test)
X_test_scaled = stdsc.transform(X_test)

In [31]:
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=79, out_features=128),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=128, out_features=2),
)

ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.01)

In [32]:
x_for_nn =  torch.from_numpy(X_train_scaled).float()
x_for_nn.shape

torch.Size([15619872, 79])

In [33]:
y_for_nn = torch.from_numpy(y_train.to_numpy())
y_for_nn.shape

torch.Size([15619872])

In [34]:
x_for_nn_test =  torch.from_numpy(X_test_scaled).float()
x_for_nn_test.shape

torch.Size([613071, 79])

In [35]:
y_for_nn_test = torch.from_numpy(y_test.to_numpy())
y_for_nn_test.shape

torch.Size([613071])

In [37]:
kl_weight = 0.1
for step in range(100):
    pre = model(x_for_nn)
    ce = ce_loss(pre, y_for_nn)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if step%10 == 0:
        print(f'Epoch: {step}/100.............')

    
_, predicted = torch.max(pre.data, 1)
print(f'F1: {f1_score(y_for_nn, predicted)}')
print(f'Accuracy: {accuracy_score(y_for_nn, predicted)}')
print(f'Precision: {precision_score(y_for_nn, predicted)}')
print(f'Recall: {recall_score(y_for_nn, predicted)}')

Epoch: 0/100.............
Epoch: 10/100.............
Epoch: 20/100.............
Epoch: 30/100.............
Epoch: 40/100.............
Epoch: 50/100.............
Epoch: 60/100.............
Epoch: 70/100.............
Epoch: 80/100.............
Epoch: 90/100.............
F1: 0.9277979108427403
Accuracy: 0.8867356915600845
Precision: 0.9830814546721328
Recall: 0.878401064316795


In [55]:
for i in range(iterations):
    pre_test = model(x_for_nn_test)
    

_, predicted_test = torch.max(pre_test.data, 1)

In [56]:
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.8626468822694336
Accuracy: 0.7637484076069493
Precision: 0.8912994487567142
Recall: 0.835779125321573
